In [1]:
import pyodbc
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv('./MRV Data - Curated_info_2020.csv')
data.head()

IMO Number           Name         Ship type  Reporting Period  \
0     6602898  OCEAN MAJESTY    Passenger ship              2020   
1     6703343       EQUALITY  Other ship types              2020   
2     7037806    IONIAN STAR       Ro-pax ship              2020   
3     7043843          TALOS        Ro-ro ship              2020   
4     7128332       SEA WIND       Ro-pax ship              2020   

    Technical efficiency  Total fuel consumption [m tonnes]  \
0  EIV (31.73 gCO₂/t·nm)                             951.37   
1  EIV (57.84 gCO₂/t·nm)                             225.70   
2   EIV (19.4 gCO₂/t·nm)                            1543.32   
3  EIV (48.71 gCO₂/t·nm)                            2625.33   
4   EIV (9.29 gCO₂/t·nm)                            4152.78   

   Total CO₂ emissions [m tonnes]  \
0                         2985.13   
1                          706.78   
2                         4947.88   
3                         8416.81   
4                        13137.92   

  Annual average Fuel consumption per distance [kg / n mile]  \
0                                             132.07           
1                                              40.75           
2                                             103.93           
3                                              64.28           
4                                              86.64           

  Annual average CO₂ emissions per distance [kg CO₂ / n mile]  \
0                                              414.4            
1                                             127.62            
2                                             333.19            
3                                             206.09            
4                                             274.09            

  Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]  \
0                                                NaN                                   
1                                              67.66                                   
2                                                NaN                                   
3                                             168.74                                   
4                                                NaN                                   

  Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]  
0                                                NaN                                   
1                                             211.88                                   
2                                                NaN                                   
3                                             540.98                                   
4                                                NaN

In [3]:
data = data[(data['Ship type']=='Oil tanker') 
            &(data['Total fuel consumption [m tonnes]'] > 0) 
            &(data['Total CO₂ emissions [m tonnes]'] > 0)]
data['Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]'] =(
    data['Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]']
    .apply(lambda x: np.nan if x=='Division by zero!' else x))
data['Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]'] = (
    data['Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]']
    .apply(lambda x: np.nan if x=='Division by zero!' else x))
data.rename(columns= {'IMO Number' : 'IMO'}, inplace=True)


data.dropna(subset=['Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]', 
                    'Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]'], inplace = True)

#float 
data['Annual average Fuel consumption per distance [kg / n mile]'] = data['Annual average Fuel consumption per distance [kg / n mile]'].astype(float)
data['Annual average CO₂ emissions per distance [kg CO₂ / n mile]'] = data['Annual average CO₂ emissions per distance [kg CO₂ / n mile]'].astype(float)

data.head()

IMO                    Name   Ship type  Reporting Period  \
249  8905787                   IRINI  Oil tanker              2020   
345  9010929                 ZIRCONE  Oil tanker              2020   
613  9125279                NORMANNA  Oil tanker              2020   
744  9145437  CASTILLO DE MONTERREAL  Oil tanker              2020   
782  9150614        HAI GONG YOU 306  Oil tanker              2020   

      Technical efficiency  Total fuel consumption [m tonnes]  \
249   EIV (10.4 gCO₂/t·nm)                            2216.30   
345  EIV (20.45 gCO₂/t·nm)                            1900.10   
613  EIV (15.36 gCO₂/t·nm)                            2088.77   
744   EIV (7.64 gCO₂/t·nm)                            3617.50   
782  EIV (12.59 gCO₂/t·nm)                            1220.84   

     Total CO₂ emissions [m tonnes]  \
249                         7050.58   
345                         5917.99   
613                         6550.71   
744                        11345.81   
782                         3840.79   

     Annual average Fuel consumption per distance [kg / n mile]  \
249                                              84.58            
345                                             123.98            
613                                              82.97            
744                                             121.91            
782                                              77.04            

     Annual average CO₂ emissions per distance [kg CO₂ / n mile]  \
249                                             269.07             
345                                             386.14             
613                                             260.19             
744                                             382.36             
782                                             242.37             

    Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]  \
249                                              15.42                                   
345                                               28.1                                   
613                                               10.3                                   
744                                              11.14                                   
782                                               7.97                                   

    Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]  
249                                              49.07                                   
345                                              87.51                                   
613                                               32.3                                   
744                                              34.93                                   
782                                              25.08

In [4]:
columns = ['IMO', 'Main Engine RPM','Deadweight', 
           'Gross Rated Tonnage', 'YearBuilt']

vessel_info = pd.DataFrame(columns=columns)

In [5]:
imolist = data.IMO.tolist()
print(len(imolist))

1835


In [6]:
### Was connecting to the databases to get some missing info. Code deleted

In [7]:
coma_list = ','.join([str(i) for i in imolist])
vessel_info = get_vessel_info(coma_list)

In [80]:
avg_cons_per_distance = data[['IMO', 'Annual average Fuel consumption per distance [kg / n mile]']]
avg_cons_per_distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1835 entries, 249 to 11823
Data columns (total 2 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   IMO                                                         1835 non-null   int64  
 1   Annual average Fuel consumption per distance [kg / n mile]  1835 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 43.0 KB


In [9]:
vessel_info_1 = pd.merge(vessel_info, avg_cons_per_distance, how = 'left', on=['IMO'])

In [10]:
vessel_info_1.dropna(subset=['MainEngineRPM'], inplace = True)

Linear Regression

In [11]:
#y = data['Annual average Fuel consumption per distance [kg / n mile]'].values
y = vessel_info_1['Annual average Fuel consumption per distance [kg / n mile]'].values

x = vessel_info_1.drop(['Annual average Fuel consumption per distance [kg / n mile]', 'IMO'], axis = 1).values


In [74]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)


In [75]:

from sklearn.linear_model import LinearRegression

ml = LinearRegression()
ml.fit(x_train, y_train)

LinearRegression()

In [76]:
y_pred_linear = ml.predict(x_test)
#print (y_pred)

In [77]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred_linear)

0.5457557615332483

In [79]:
#values between 0.2 and 0.5 shows that the model can relatively predict the data accurately.
from sklearn.metrics import mean_squared_error
lin_rmse = np.sqrt(mean_squared_error(y_test, y_pred_linear))
lin_rmse

34.964988026501224

In [73]:
pred_y_linear_df = pd.DataFrame({'Actual Value': y_test, 'Predicted Value':y_pred_linear, 'Difference': y_test - y_pred_linear})
pred_y_linear_df[0:30]

Actual Value  Predicted Value  Difference
0         141.58       145.408998   -3.828998
1         105.74       117.539994  -11.799994
2          90.78       115.506963  -24.726963
3         173.99       159.954056   14.035944
4         138.28       122.703559   15.576441
5         135.30       154.749448  -19.449448
6         140.70       149.988896   -9.288896
7         123.75       114.584109    9.165891
8         160.29       183.050451  -22.760451
9          78.75        83.286465   -4.536465
10        107.08       123.422881  -16.342881
11        239.34       179.239790   60.100210
12        208.78       168.126371   40.653629
13        203.95       316.270415 -112.320415
14        160.69       161.169315   -0.479315
15        155.74       155.778679   -0.038679
16        209.28       160.297912   48.982088
17        149.80       175.081537  -25.281537
18        110.59       117.634918   -7.044918
19        232.60       307.145344  -74.545344
20        126.70       141.483879  -14.783879
21        148.08       157.356798   -9.276798
22         94.55       116.322399  -21.772399
23        156.99       137.916405   19.073595
24        152.41       136.048990   16.361010
25        136.94       116.898331   20.041669
26        175.58       151.372170   24.207830
27        159.45       148.432231   11.017769
28        345.80       281.036899   64.763101
29         97.74       132.493668  -34.753668

Ridge

In [62]:
from sklearn.linear_model import Ridge

# define model
model = Ridge(alpha=1.0)
# fit model
model.fit(x_train, y_train)


Ridge()

In [63]:
y_pred_ridge = model.predict(x_test)

In [64]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred_ridge)

0.38316361222169903

In [113]:
pred_y_ridge_df = pd.DataFrame({'Actual Value': y_test, 'Predicted Value':y_pred_ridge, 'Difference': y_test - y_pred_ridge})
pred_y_ridge_df[0:30]

Actual Value  Predicted Value  Difference
0         163.06       144.837497   18.222503
1         110.71       129.544433  -18.834433
2         167.80       178.828273  -11.028273
3         220.31       237.466199  -17.156199
4         174.31       183.222942   -8.912942
5         140.59       142.962492   -2.372492
6         114.65       133.580893  -18.930893
7         106.34       102.185960    4.154040
8         159.63       149.780623    9.849377
9         121.92       121.496212    0.423788
10        173.13       173.175627   -0.045627
11        149.53       122.988482   26.541518
12        152.70       166.317847  -13.617847
13        139.56       138.257359    1.302641
14         38.52        89.339866  -50.819866
15        136.60       146.848259  -10.248259
16         79.29       139.206019  -59.916019
17         60.71        75.349622  -14.639622
18        176.27       167.371097    8.898903
19        125.70       131.876473   -6.176473
20         93.52       108.192331  -14.672331
21        204.11       187.634950   16.475050
22        234.27       275.783220  -41.513220
23        185.28       164.519071   20.760929
24        157.23       144.346223   12.883777
25        131.70       125.350274    6.349726
26        161.29       134.724960   26.565040
27        246.05       265.097508  -19.047508
28        167.00       152.971010   14.028990
29        202.95       248.833014  -45.883014

Lasso

In [51]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha=1.0)
model_lasso.fit(x_train, y_train)

C:\Users\p.alexias\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1819655.4103837656, tolerance: 544.2952938784987
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [52]:
y_pred_lasso = model_lasso.predict(x_test)

r2_score(y_test, y_pred_lasso)

0.4074784314586175

In [108]:
pred_y_lasso_df = pd.DataFrame({'Actual Value': y_test, 'Predicted Value':y_pred_lasso, 'Difference': y_test - y_pred_lasso})
pred_y_lasso_df[0:30]

Actual Value  Predicted Value  Difference
0         163.06       144.832358   18.227642
1         110.71       129.172281  -18.462281
2         167.80       178.721835  -10.921835
3         220.31       237.597310  -17.287310
4         174.31       183.062849   -8.752849
5         140.59       143.368671   -2.778671
6         114.65       134.109848  -19.459848
7         106.34       102.524259    3.815741
8         159.63       149.801953    9.828047
9         121.92       121.545261    0.374739
10        173.13       172.790706    0.339294
11        149.53       123.258235   26.271765
12        152.70       165.966598  -13.266598
13        139.56       138.326646    1.233354
14         38.52        89.609715  -51.089715
15        136.60       146.604596  -10.004596
16         79.29       138.872957  -59.582957
17         60.71        75.711675  -15.001675
18        176.27       167.107036    9.162964
19        125.70       132.046468   -6.346468
20         93.52       108.244342  -14.724342
21        204.11       187.521526   16.588474
22        234.27       275.322768  -41.052768
23        185.28       164.281910   20.998090
24        157.23       144.272025   12.957975
25        131.70       125.154594    6.545406
26        161.29       134.948041   26.341959
27        246.05       265.031732  -18.981732
28        167.00       152.583822   14.416178
29        202.95       248.781740  -45.831740

Decision tree

In [119]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree = dtree.fit(x_train, y_train)

ValueError: Unknown label type: 'continuous'

In [ ]:
y_pred_tree = tree.predict(x_test)

r2_score(y_test, y_pred_tree)